In [294]:
import numpy as np

In [540]:
tiles = []
tilenumbers = []

with open('input.txt') as f:
    state = 'name'
    tile = []
    tiles.append(tile)
    
    for line in f:
        line = line.strip()
        if not line:
            state = 'name'
            tile = []
            tiles.append(tile)
        elif state == 'name':
            tilenumber = int(line.split()[1].rstrip(':'))
            tilenumbers.append(tilenumber)
            state = 'tile'
        elif state == 'tile':
            tile.append(list(line))           
        

In [541]:
dim = int(np.sqrt(len(tiles)))
dim

12

In [542]:
tiles = [np.array(tile) for tile in tiles]

In [543]:
tilesbynumber = dict(zip(tilenumbers, tiles))

In [544]:
from itertools import product

In [545]:
start = numpy.empty((dim, dim))

In [546]:
availabletiles = set(tilenumbers)

In [547]:
checkoffset = {
    'U': (-1, 0),
    'D': (1, 0),
    'L': (0, -1),
    'R': (0, 1)
}

In [548]:
def gettile(tilenumber, flip, rot):
    tile = np.rot90(tilesbynumber[tilenumber], k=rot)
    if flip:
        tile = np.fliplr(tile)
    return tile

# Working solution, too slow on real data

In [549]:
def layout(tilearray, availabletiles):
#     print(availabletiles)
#     print(tilearray)
    i = 0
    j = 0
    arrangement = [[None for i in range(dim)] for j in range(dim)]
    
    for tilenumber, flip, rot in tilearray:
        tile = gettile(tilenumber, flip, rot)
        
        arrangement[i][j] = tile
        j += 1
        if j == dim:
            i += 1
            j = 0
        
    for i, j in product(range(dim), repeat=2):
        if arrangement[i][j] is None:
            continue
            
        tile = arrangement[i][j]
        
        checks = []
        if i > 0:
            checks.append('U')
        if i < dim - 1:
            checks.append('D')
        if j > 0:
            checks.append('L')
        if j < dim - 1:
            checks.append('R')
        
        for check in checks:
#             print(i, j)
#             print(check)
            otheri, otherj = checkoffset[check]
    
            othertile = arrangement[i + otheri][j + otherj]
            
            if othertile is None:
                continue
            
#             print(i, j, otheri, otherj)
#             print(check)
#             print(tile)
#             print(othertile)
            
            if check == 'U':
                if not (othertile[-1, :] == tile[0, :]).all():
                    return False
            if check == 'D':
                if not (othertile[0, :] == tile[-1, :]).all():
                    return False
            if check == 'L':
                if not (othertile[:, -1] == tile[:, 0]).all():
                    return False
            if check == 'R':
                if not (othertile[:, 0] == tile[:, -1]).all():
                    return False

    if not availabletiles:
        return tilearray

    for tilenumber in availabletiles:
        for flip in (True, False):
            for rot in range(4):
                result = layout(tilearray + [(tilenumber, flip, rot)], availabletiles - {tilenumber})
                if result:
                    return result

    return False

In [307]:
solution = [1951, 2311, 3079, 
            2729, 1427, 2473,
            2971, 1489, 1171]

In [308]:
def printtile(tilenumber, flip, rot):
    print('\n'.join(''.join(c for c in row) for row in gettile(tilenumber, flip, rot)))

In [309]:
arrangement = [
(1951, True, 2),
(2311, True, 2),
(3079, False, 0),
(2729, True, 2),
(1427, True, 2),
(2473, True, 1),
(2971, True, 2),
(1489, True, 2),
(1171, True, 0),
]

In [310]:
layout(arrangement, [])

[(1951, True, 2),
 (2311, True, 2),
 (3079, False, 0),
 (2729, True, 2),
 (1427, True, 2),
 (2473, True, 1),
 (2971, True, 2),
 (1489, True, 2),
 (1171, True, 0)]

In [311]:
workinglayout = layout([], availabletiles)

In [312]:
availabletiles

{1171, 1427, 1489, 1951, 2311, 2473, 2729, 2971, 3079}

# Try to go faster

In [550]:
from collections import defaultdict

In [551]:
leftedges = defaultdict(set)
topedges = defaultdict(set)

In [552]:
for tilenumber, tile in zip(tilenumbers, tiles):
    for rot in range(4):
        for flip in (True, False):
            newtile = np.rot90(tile, rot)
            if flip:
                newtile = np.fliplr(newtile)
                
            leftedge = ''.join(newtile[:, 0])
            topedge = ''.join(newtile[0, :])
            leftedges[leftedge].add((tilenumber, flip, rot))
            topedges[topedge].add((tilenumber, flip, rot))

In [553]:
i, j = 0, 0

In [554]:
available = {(tilenumber, flip, rot) 
             for tilenumber in tilenumbers 
             for flip in (True, False)
             for rot in range(4) 
            }

In [555]:
len(available)

1152

In [583]:
def build_layout(layout_tiles, layout_flip, layout_rot, i, j, available):
    def triple(i, j):
        return layout_tiles[i, j], layout_flip[i, j], layout_rot[i, j]

    if not np.isnan(layout_tiles).any():
        return layout_tiles, layout_flip, layout_rot
    
    possibletiles = available.copy()
    if j > 0:
        neighbour_left = gettile(*triple(i, j-1))
        left_edge = ''.join(neighbour_left[:, -1])
        possibletiles &= leftedges[left_edge]
    if i > 0:
        neighbour_above = gettile(*triple(i-1, j))
        top_edge = ''.join(neighbour_above[-1, :])
        possibletiles &= topedges[top_edge]
            
    if len(possibletiles) == 0:
        return False
        
    for tile, flip, rot in possibletiles:
        new_layout_tiles = layout_tiles.copy()
        new_layout_rot = layout_rot.copy()
        new_layout_flip = layout_flip.copy()

        new_layout_tiles[i, j] = tile
        new_layout_rot[i, j] = rot
        new_layout_flip[i, j] = flip
        
        new_available = available - {
            (tile, f, r) for f in (True, False) for r in range(4)}

        next_j = j + 1
        next_i = i
        if next_j == dim:
            next_j = 0
            next_i = i + 1
        
        result = build_layout(
            new_layout_tiles, 
            new_layout_flip, 
            new_layout_rot, 
            next_i, 
            next_j, 
            new_available)
        if result is not False:
            return result

    return False

In [584]:
layout_tiles = np.tile(np.nan, (dim, dim))
layout_flip = np.tile(False, (dim, dim))
layout_rot = np.tile(0, (dim, dim))

In [585]:
# i = 0
# j = 0
# for tile, flip, rot in arrangement[:-2]:
#     layout_tiles[i, j] = tile
#     layout_flip[i, j] = flip
#     layout_rot[i, j] = rot
#     j += 1
#     if j == dim:
#         j = 0
#         i += 1

In [587]:
layout_tiles, layout_flip, layout_rot = build_layout(layout_tiles, layout_flip, layout_rot, 0, 0, available)

In [588]:
layout_tiles[0, 0]*layout_tiles[0, -1]*layout_tiles[-1, 0]*layout_tiles[-1, -1]

79412832860579.0

# Part 2

In [575]:
monster_str = """                  # 
#    ##    ##    ###
 #  #  #  #  #  #   """  

In [582]:
monster_array = np.array([list(line) for line in monster_str.split('\n')])
monster = (monster_array == '#').astype(int)

In [596]:
image = []
for i in range(dim):
    row = []
    image.append(row)
    for j in range(dim):
        tile = gettile(layout_tiles[i, j], layout_flip[i, j], layout_rot[i, j])[1:-1, 1:-1]
        row.append(tile)
    

image = (np.block(image) == '#').astype('int')

In [602]:
import matplotlib.pyplot as plt

In [598]:
image.shape

(96, 96)

In [600]:
monster.shape

(3, 20)

In [606]:
import scipy.signal

In [615]:
for rot in range(4):
    for flip in (True, False):
        flipped_image = np.rot90(image, rot)
        if flip:
            flipped_image = np.fliplr(flipped_image)
            
        monsters = (scipy.signal.convolve2d(flipped_image, monster) == monster.sum()).sum()
        if monsters:
            print(monsters)

25


In [617]:
image.sum() - 25*monster.sum()

2155